In [1]:
from selenium import webdriver
import datetime
import time
from selenium.common.exceptions import NoSuchElementException, TimeoutException, StaleElementReferenceException
from scrapy.selector import Selector
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
import os
import re
import shutil
import xlrd
import pymysql
from scrapy.selector import Selector
import re
import random



def get_element(browser, by=By.XPATH, value="//body", times=3, timeout=8, poll_frequency=0.3, multi=False, check=False):
        """获取元素 v2.9.20170223
        @param browser: 浏览器对象
        @param by: 搜索元素方式
        @param value: 搜索元素方式的值
        @param times: 重试次数
        @param timeout: 超时秒数
        @param poll_frequency: 调用频率
        @param multi: 是否是获取多个元素，True-是，False-否
        @param check: 是否需要检查WebElement元素的有效性(位置不为0，尺寸不为0)，True-需要，False-不需要
        @return: 获取到的元素"""
        
        # 重试次数
        try_times = 1
        # 元素
        if multi:
            element = []
        else:
            element = None
        # 定位器
        locator = (by, value)
        
        wait_seconds = 0  # 等待秒数
        start_time = datetime.datetime.now()  # 开始时间
        end_time = datetime.datetime.now()  # 结束时间
        
        # 循环等待加载元素
        while not element and try_times <= times and wait_seconds <= timeout:
            # 元素未找到 并且 重试次数小于规定次数 并且 在超时范围内
            end_time = datetime.datetime.now()
            interval = end_time - start_time
            wait_seconds = interval.total_seconds()
            print("[BrowserHelper :: get_element]", "加载WebElement元素 ...", "[try_times] =", try_times, "[wait_seconds] =", wait_seconds, "[locator] =", locator)
            try:
                # 元素加载等待
                call = multi and EC.presence_of_all_elements_located or EC.presence_of_element_located
                webdirver_wait = WebDriverWait(driver=browser, timeout=timeout, poll_frequency=poll_frequency, ignored_exceptions=None)
                element = webdirver_wait.until(call(locator))
                
                # 检查WebElement元素的有效性(位置不为0，尺寸不为0)
                if check:
                    if element and element.location and element.location.get('x', 0) > 0 and element.location.get('y', 0) and \
                        element.size and element.size.get('width', 0) > 0 and element.size.get('height', 0) > 0 :
                        # WebElement元素有效
                        pass
                    else:
                        element = multi and [] or None
            
            except NoSuchElementException as e:
                print("[BrowserHelper :: get_element] No such element: %s = %s" % locator, e)
            except TimeoutException as e:
                print("[BrowserHelper :: get_element] load element: %s = %s timeout" % locator, e)
            
            print("[BrowserHelper :: get_element] waite for load element ...", "[try_times] =", try_times, "[element] =", (element and True or False))
            
            try_times += 1
        
        print("[BrowserHelper :: get_element] get element(%s = %s) %s!" % (by, value, element and "successfull" or "failed"))
        
        return element


def open_excel(excel_file):
    try:
        book = xlrd.open_workbook(excel_file)  #文件名，把文件与py文件放在同一目录下
    except:
        print("open excel file failed!")
    try:
        sheet = book.sheet_by_name(r"【生意参谋平台】1")   #execl里面的worksheet1
        return sheet
    except:
        print("locate worksheet in excel failed!")


def connect():
    #连接数据库
    try:
        db = pymysql.connect(host="192.168.0.23",user="root",
            passwd="njds@2019",
            db="bi",
            charset='utf8')
        
#         db = pymysql.connect(host="localhost",user="root",
#             passwd="aixocm123",
#             db="bi",
#             charset='utf8')
        
        return db
    except:
        print("could not connect to mysql server")


def init_chrome_browser():
    # 启动允许多个文件下载的浏览器
    options = webdriver.ChromeOptions()
    prefs = {"credentials_enable_service": False, "profile.password_manager_enabled": False, "profile.managed_default_content_settings.images": 1,'profile.default_content_settings.popups': 0,  'download.default_directory': r'c:\Downloads4', "profile.default_content_setting_values.automatic_downloads":1}
    options.add_experimental_option('prefs', prefs)
    executable_path = "C:\Program Files (x86)\Google\Chrome\Application\chromedriver.exe"
    options.add_argument('user-agent="Mozilla/5.0 (Linux; U; Android 8.1.0; zh-cn; BLA-AL00 Build/HUAWEIBLA-AL00) AppleWebKit/537.36 (KHTML, like Gecko) Version/4.0 Chrome/57.0.2987.132 MQQBrowser/8.9 Mobile Safari/537.36"')
    options.add_experimental_option('excludeSwitches', ['enable-automation'])
    options.add_argument('--user-data-dir=C:\\Users\\xieruiping\\AppData\\Local\\Google\\Chrome\\User Data')
#     options.add_extension("C:\\Users\\xieruiping\\Desktop\\extension_0_0_1_1.crx");
    browser = webdriver.Chrome(chrome_options=options)
    browser.maximize_window()
    return browser

# executable_path=executable_path,
def clear_html_re(src_html):
    '''
    正则清除HTML标签
    :param src_html:原文本
    :return: 清除后的文本
    '''
    content = re.sub(r"</?(.+?)>", "", src_html) # 去除标签
    # content = re.sub(r"&nbsp;", "", content)
    dst_html = re.sub(r"\s+", "", content)  # 去除空白字符
    return dst_html


def scroll_to(browser, x=0, y=0):
        """浏览器内容水平滚动到指定位置x,垂直滚动到指定位置y
        @param browser: 浏览器实例对象
        @param x: 水平滚动到的位置x
        @param y: 垂直滚动到的位置y"""
        script = """
            (function() {
                window.scroll(%(x)s, %(y)s);
            })();""" % {'x': str(x), 'y': str(y), }
        try:
            browser.execute_script(script=script)
        except Exception as e:
            print("[scroll] Browser scroll to (%s, %s) failed" % (x, y))


def scroll_to_bottom(browser, step=100, delay=1000, currentY=0):
        """浏览器内容垂直滚动到最底部
        @param browser: 浏览器实例对象
        @param step: 滚动步长
        @param delay: 滚动延迟时间
        @param currentY: 当前垂直滚动到的位置"""
        
        script = """
            (function () {
                var y = %(currentY)s;
                var step = %(step)s;
                window.scroll(0, y);
                
                function f() {
                    if (y < document.body.scrollHeight) {
                        y += step;
                        window.scroll(0, y);
                        setTimeout(f, %(delay)s);
                    } else {
                        window.scroll(0, %(position)s);
                        document.title += "scroll-done";
                    }
                }
                
                setTimeout(f, %(delay)s);
            })();
        """ % {'step': step, 'delay': delay, 'currentY': currentY, 'position': "document.body.scrollHeight", }
        
        try:
            browser.execute_script(script=script)
        except Exception as e:
            print("[scroll] Browser scroll bottom failed" % step)


def get_scroll_range(browser):
        """获取浏览器可滚动范围
        @param browser: 浏览器实例对象
        @return: 浏览器滚动横向、纵向滚动范围"""
        # 获取浏览器滚动距离
        js = "var scrollWidth = document.body.scrollWidth; return(scrollWidth);"
        scroll_width = browser.execute_script(js)
        js = "var scrollHeight = document.body.scrollHeight; return(scrollHeight);"
        scroll_height = browser.execute_script(js)
        return (scroll_width, scroll_height,)


def scroll(browser, step=100, delay=1000, is_half=False):
    """浏览器内容垂直滚动
    @param browser: 浏览器实例对象
    @param step: 滚动步长
    @param delay: 滚动延迟时间
    @param is_half: 是否滚动到屏幕内容一半的位置，True-是，False-否"""

    script = """
        (function () {
            var y = 0;
            var step = %(step)s;
            window.scroll(0, 0);

            function f() {
                if (y < document.body.scrollHeight) {
                    y += step;
                    window.scroll(0, y);
                    setTimeout(f, %(delay)s);
                } else {
                    
                    document.title += " scroll-done";
                }
            }

            setTimeout(f, %(delay)s);
        })();
    """ % {'step': step, 'delay': delay}
    
    try:
        browser.execute_script(script=script)
    except Exception as e:
            print("[scroll] Browser scroll %s failed" % step)


def get_real_url(url, response):
    if not url.startswith('http'):
        base_url = get_base_url(response)
        url = urljoin_rfc(base_url, url)
        return url

# import win32api
# win32api.ShellExecute(0, "open", r"D:\install\fiddler4\Fiddler\Fiddler.exe", "", "", 1)

# browser = init_chrome_browser()

# browser.get("https://sycm.taobao.com/")
# browser.implicitly_wait(50)

# # 关闭fidder

In [3]:
db = connect()
cursor = db.cursor()
browser = init_chrome_browser()
# browser = webdriver.Chrome()

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:125: DeprecationWarning: use options instead of chrome_options


In [ ]:
# 打开fiddler
# import win32api
# win32api.ShellExecute(0, "open", r"C:\Users\NJDS\AppData\Local\Programs\Fiddler\Fiddler.exe", "", "", 1)

yesterday = datetime.date.today() + datetime.timedelta(days = -1)    # 昨天日期
today = datetime.date.today()
tomorrow = datetime.date.today() + datetime.timedelta(days = 1) 

mall_id_dict = {}
with open("pdd_mall.txt", mode="r", encoding="utf-8") as f:
    for line in f.readlines():
        line = line.replace("\n", "").strip().replace("\t", '')
        key = line.split("(店铺和店铺id)")[0]
        value = line.split("(店铺和店铺id)")[1]
        mall_id_dict[key] = value



value1_list = []
for mall_id in mall_id_dict.keys():
    browser.refresh()
    browser.get("http://mobile.yangkeduo.com/mall_page.html?mall_id=%s" % mall_id)
    browser.implicitly_wait(50)
    
    
    page_source = browser.page_source
    # print("page_source =", page_source)
    selector = Selector(text=page_source)
    
    store_name = selector.xpath("//div[@class='mall-title']/span/text()").extract() and selector.xpath("//div[@class='mall-title']/span/text()").extract()[0] or mall_id_dict.get(mall_id) 
    
    cursor.execute("select distinct goods_id from ds_pdd_mall_goods_id_d where mall_id=%s" % mall_id)
    results = cursor.fetchall()
    print("mall_id", mall_id, "results=", results)
    for result in list(results):
        goods_id = result[0]
        print(result[0])
        browser.get("http://mobile.yangkeduo.com/goods.html?goods_id=%s" % result[0])
        browser.implicitly_wait(50)
        time.sleep(random.uniform(1,1.5))
        
        try:
            page_source = browser.page_source
            # print("page_source =", page_source)
            selector = Selector(text=page_source)

            count_date = today
            store_name = store_name
            goods_pic_img = selector.xpath("//ul[@class='islider-outer']//img")

            goods_pic = ','.join([img.xpath("./@src").extract()[0] for img in goods_pic_img])
            goods_name = selector.xpath("//span[@class='enable-select']/text()").extract() and selector.xpath("//span[@class='enable-select']/text()").extract()[0] or ""
            alone_goods_price = selector.xpath("//div[@class='goods-direct-btn-new']/span").extract() and clear_html_re(selector.xpath("//div[@class='goods-direct-btn-new']/span").extract()[0]).replace("￥", "") or 0
            group_goods_price = selector.xpath("//div[@class='goods-group-btn-new']/span").extract() and clear_html_re(selector.xpath("//div[@class='goods-group-btn-new']/span").extract()[0]).replace("￥", "") or 0

            total_comments_number = selector.xpath("//div[@class='goods-reviews-num']/text()").extract() and selector.xpath("//div[@class='goods-reviews-num']/text()").extract()[0].replace("商品评价(", "").replace(")", "") or 0
            group_goods_num = selector.xpath("//span[@class='g-sales regular-text']/text()").extract() and selector.xpath("//span[@class='g-sales regular-text']/text()").extract()[0].replace("已拼", "").replace("件", "").replace("全网总售", "") or "0"
            if group_goods_num == "":
                group_goods_num = selector.xpath("//div[@class='s-c-left']/span/text()").extract() and selector.xpath("//div[@class='s-c-left']/span/text()").extract()[0] or 0

            if "万+" in group_goods_num:
                group_goods_num = float(group_goods_num.replace("万", "").replace("+", "")) * 10000
            elif "万" in group_goods_num:
                group_goods_num = float(group_goods_num.replace("万", "").replace("+", "")) * 10000
            elif "人想买" in group_goods_num:
                group_goods_num = 0

            comments_detail_div= selector.xpath("//ul[@id='grt-container']//span")
            comments_detail = ','.join([conment.xpath("./text()").extract()[0] for conment in comments_detail_div])
            
            value = (count_date, mall_id,  store_name, goods_id, goods_pic, goods_name, alone_goods_price, group_goods_num, group_goods_price, total_comments_number, comments_detail)
        #         value1_list.append(value)
#             print("##" * 40)
#             print("[value] =", value)
#             print("##" * 40)
            sql = """INSERT INTO ds_pdd_client_store_goods_d(count_date,mall_id,  store_name, goods_id,goods_pic, goods_name, alone_goods_price, group_goods_num, group_goods_price, total_comments_number, comments_detail) 
                   VALUES(%s, %s, %s, %s, %s, %s, %s, %s,%s,%s, %s)"""

            try:
                cursor.execute(sql, value)
            except Exception as e:
                print("获取异常")
            
            db.commit()
            time.sleep(random.uniform(2,2.5))
        except Exception as e:
            print(e)

cursor.close()
db.close()
# 手动关闭fiddler

In [14]:
browser.quit()

In [4]:
browser.get("https://mobile.yangkeduo.com")

scroll(browser=browser, step=1000, delay=100, is_half=False)

while not browser.title.count("scroll-done"):
    break

In [12]:
yesterday = datetime.date.today() + datetime.timedelta(days = -1)    # 昨天日期
today = datetime.date.today()
tomorrow = datetime.date.today() + datetime.timedelta(days = 1) 

page_source = browser.page_source
# print("page_source =", page_source)
selector = Selector(text=page_source)

value1_list = []

divs = selector.xpath("//div[@id='search-result']")
div_search_results = divs.xpath(".//div[contains(@class, '_28ee0W_v')]")
for result in div_search_results:
    count_date = today
    store_name = result.xpath(".//div[@class='_2fjEb8vX']/text()").extract() and result.xpath(".//div[@class='_2fjEb8vX']/text()").extract()[0] or "" 
    goods_pic = result.xpath(".//img[@class='agYwTmI9']/@src").extract() and result.xpath(".//img[@class='agYwTmI9']/@src").extract()[0] or ""
    goods_detail = result.xpath(".//p[@class='a6yknZ_Z']/text()").extract() and result.xpath(".//p[@class='a6yknZ_Z']/text()").extract()[0] or ""
    goods_bright = result.xpath(".//div[@class='_2Ue22KBs']").extract() and clear_html_re(result.xpath(".//div[@class='_2Ue22KBs']").extract()[0])or ""
    goods_price = result.xpath(".//p[@class='_2IX6eXBh']").extract() and clear_html_re(result.xpath(".//p[@class='_2IX6eXBh']").extract()[0]).replace("￥", "")  or ""
    group_num = result.xpath(".//p[@class='_2KLtl24M']/text()").extract() and result.xpath(".//p[@class='_2KLtl24M']/text()").extract()[0] or ""
    
    
    value = (count_date, store_name, goods_pic, goods_detail, goods_bright, goods_price, group_num)
    value1_list.append(value)
    
sql = """INSERT INTO ds_pdd_client_page_d(count_date, store_name, goods_pic, goods_detail, goods_bright, goods_price, group_num) 
           VALUES(%s,%s, %s, %s, %s, %s, %s)"""

try:
    cursor.executemany(sql, value1_list)
except Exception as e:
    print(e)

db.commit()

# print(divs.extract())


import json
page_source = browser.page_source
selector = Selector(text=page_source)
scripts = selector.xpath("//script[@id='__NEXT_DATA__']")
for script in scripts:
    data = clear_html_re(script.extract())
#     print(data)
    dict_data = json.loads(data)
#     print(dict_data)
    
    data_list = dict_data.get("props", {}).get("pageProps", {}).get("data", {})
    print(len(data_list))
#     data_list = dict_data.get("props", {}).get("pageProps", {}).get("data", {}).get("ssrListData", {}).get("list", [])
    print(data_list)
    print("###" * 40)

WebDriverException: Message: chrome not reachable
  (Session info: chrome=73.0.3683.86)
  (Driver info: chromedriver=72.0.3626.69 (3c16f8a135abc0d4da2dff33804db79b849a7c38),platform=Windows NT 6.1.7601 SP1 x86_64)


In [34]:
db = connect()
cursor = db.cursor()

scroll(browser=browser, step=1000, delay=1000, is_half=False)

while not browser.title.count("scroll-done"):
    break

time.sleep(1)
page_source = browser.page_source
selector = Selector(text=page_source)

divs = selector.xpath("//div[@id='__next']")
div_results = divs.xpath(".//div[@id='_28ee0W_v _2ENZGW3_']")
for div_result in div_results:
    print(div_result.extract())
    print("###" * 40)

# scripts = selector.xpath("//script[@id='__NEXT_DATA__']")
# for script in scripts:
#     print(script.extract())
#     print("###" * 40)



#         # 获取服务视窗的数据
#         page_source = browser.page_source.encode("utf-8")
#         # 构建选择器
#         selector1 = Selector(text=page_source)
        
        
#         # 关闭弹窗
#         if selector1.xpath("//div[@class='ebase-ImageTips__dsImageTipsWrapper']"):
#             close_popup = get_element(browser=browser, by=By.XPATH, value="//div[@class='ebase-ImageTips__dsImageTipsWrapper']//span[@class='ebase-ImageTips__dsImageTipsCloseBtn']")
#             action = ActionChains(browser)
#             action.click(close_popup).perform()
        
        
#         time.sleep(2)
#         # 获取服务视窗的数据
#         page_source = browser.page_source.encode("utf-8")
#         # 构建选择器
#         selector = Selector(text=page_source)
        
#         ranking = selector.xpath("//div[@class='month-rank main-index']//span[contains(@class, 'index-value')]/text()").extract() and selector.xpath("//div[@class='month-rank main-index']//span[contains(@class, 'index-value')]/text()").extract()[0].strip() or ""
        
#         hierarchy =  selector.xpath("//div[@class='month-rank main-index']//span[@class='detail']/text()").extract() and selector.xpath("//div[@class='month-rank main-index']//span[@class='detail']/text()").extract()[0].strip() or ""
        
#         industry = selector.xpath("//div[@class='trend-container']//p[@class='cate-name text-overflow']/text()").extract() and selector.xpath("//div[@class='trend-container']//p[@class='cate-name text-overflow']/text()").extract()[0].replace("行业：", '').strip() or ""
        
#         print("##" * 40)
#         print("[ranking] =", ranking)
#         print("[hierarchy] =", hierarchy)
#         print("[industry] =", industry)
#         print("##" * 40)
        
        
#         # 点击服务视窗
#         element = get_element(browser=browser, by=By.XPATH, value="//div[@class='ebase-Switch__root op-ebase-switch']/span[2]")
#         js_click = """arguments[0].click();"""
#         browser.execute_script(js_click, element)
        
        
#         # 浏览器的获取数据节点
#         ele_shichuang = get_element(browser=browser, by=By.XPATH, value="//div[@id='floor-service-experience']")
#         x, y = ele_shichuang.location.get("x"), ele_shichuang.location.get("y")
        
#         # x, y = get_scroll_range(browser)
#         scroll_to(browser,y=y)
        
#         time.sleep(2)
#         # 获取服务视窗的数据
#         page_source = browser.page_source.encode("utf-8")
#         # 构建选择器
#         selector2 = Selector(text=page_source)
        
#         if selector2.xpath("//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper']"):
            
#             # 需要鼠标放入到区域内
#             polygon_buttons = get_element(browser=browser, by=By.XPATH, value="//div[@class='componentsServiceExperienceBoard__root']//div[@class='componentsServiceExperienceBoard__radarChart']", multi=True)
#             for polygon_button in polygon_buttons:
#                 action = ActionChains(browser)
                
# #                 x, y, width, heigth = polygon_button.location.get('x'), polygon_button.location.get('y'), polygon_button.size.get('width'), polygon_button.size.get('height')
#                 action.move_to_element(polygon_button).perform()
#                 time.sleep(1)
        
#         # 获取服务视窗的数据
#         page_source = browser.page_source.encode("utf-8")
#         # 构建选择器
#         selector = Selector(text=page_source)
        
        
#         store_name = store_name
#         count_date = selector.xpath("//div[@class='oui-date-picker-current-date']").extract() and selector.xpath("//div[@class='oui-date-picker-current-date']").extract()[0] or ""
#         count_date = clear_html_re(count_date).replace("统计时间", "")
        

#         xpath_wyg = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@value='wygDsrRmkScore']//div[@class='oui-indexCell-indexValue oui-num']/text()"
#         wyg_integration_score = selector.xpath(xpath_wyg).extract() and selector.xpath(xpath_wyg).extract()[0] or 0.00
#         xpath_is_stand = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='oui-indexCell-indexName']//span[contains(@class, 'componentsServiceExperienceBoard')]/text()"
#         wyg_is_standard = selector.xpath(xpath_is_stand).extract() and selector.xpath(xpath_is_stand).extract()[0] or ""
        
#         print("##" * 40)
#         print("[count_date] =", count_date)
#         print("[wyg_integration_score] =", wyg_integration_score)
#         print("[wyg_is_standard] =", wyg_is_standard)
#         print("##" * 40)
        
#         xpath_market = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='oui-indexCell-indexValue oui-num']/text()"
#         market_comprehensive_score = selector.xpath(xpath_market).extract() and selector.xpath(xpath_market).extract()[0] or 0.00
#         xpath_is_market = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='oui-indexCell-indexName']//span[contains(@class, 'componentsServiceExperienceBoard')]/text()"
#         market_is_standard = selector.xpath(xpath_is_market).extract() and selector.xpath(xpath_is_market).extract()[0] or ""
        
#         print("##" * 40)
#         print("[market_comprehensive_score] =", market_comprehensive_score)
#         print("[market_is_standard] =", market_is_standard)
#         print("##" * 40)
        
#         xpath1 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//ul[@class='ebase-TooltipContent__legendContentList']/li[4]/p[@class='ebase-TooltipContent__value']/text()"
#         xpath1_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='recharts-wrapper']//g[@class='recharts-layer recharts-polar-angle-axis']//g[@class='recharts-layer recharts-polar-angle-axis-ticks']/g[@class='recharts-layer recharts-polar-angle-axis-tick'][1]//text[contains(@class, 'componentsRadarTick')]/text()"
        
#         # 商品体验
#         commodity_score = selector.xpath(xpath1).extract() and selector.xpath(xpath1).extract()[0] or 0
#         commodity_is_standard = selector.xpath(xpath1_2).extract() and selector.xpath(xpath1_2).extract()[0] or ""


#         xpath2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//ul[@class='ebase-TooltipContent__legendContentList']/li[3]/p[@class='ebase-TooltipContent__value']/text()"
#         xpath2_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='recharts-wrapper']//g[@class='recharts-layer recharts-polar-angle-axis']//g[@class='recharts-layer recharts-polar-angle-axis-ticks']/g[@class='recharts-layer recharts-polar-angle-axis-tick'][2]//text[contains(@class, 'componentsRadarTick')]/text()"

#         # 投诉体验
#         complaints_score = selector.xpath(xpath2).extract() and selector.xpath(xpath2).extract()[0] or 0
#         complaints_is_standard = selector.xpath(xpath2_2).extract() and selector.xpath(xpath2_2).extract()[0] or ""


#         xpath3 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//ul[@class='ebase-TooltipContent__legendContentList']/li[2]/p[@class='ebase-TooltipContent__value']/text()"
#         xpath3_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='recharts-wrapper']//g[@class='recharts-layer recharts-polar-angle-axis']//g[@class='recharts-layer recharts-polar-angle-axis-ticks']/g[@class='recharts-layer recharts-polar-angle-axis-tick'][3]//text[contains(@class, 'componentsRadarTick')]/text()"

#         # 物流体验
#         logistics_score = selector.xpath(xpath3).extract() and selector.xpath(xpath3).extract()[0] or 0
#         logistics_is_standard = selector.xpath(xpath3_2).extract() and selector.xpath(xpath3_2).extract()[0] or ""

#         xpath4 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//ul[@class='ebase-TooltipContent__legendContentList']/li[1]/p[@class='ebase-TooltipContent__value']/text()"
#         xpath4_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='recharts-wrapper']//g[@class='recharts-layer recharts-polar-angle-axis']//g[@class='recharts-layer recharts-polar-angle-axis-ticks']/g[@class='recharts-layer recharts-polar-angle-axis-tick'][4]//text[contains(@class, 'componentsRadarTick')]/text()"

#         # 售后体验
#         aftersale_score = selector.xpath(xpath4).extract() and selector.xpath(xpath4).extract()[0] or 0
#         aftersale_is_standard = selector.xpath(xpath4_2).extract() and selector.xpath(xpath4_2).extract()[0] or ""

#         print("##" * 40)
#         print("[commodity_score] =", commodity_score)
#         print("[commodity_is_standard] =", commodity_is_standard)
#         print("[complaints_score] =", complaints_score)
#         print("[complaints_is_standard] =", complaints_is_standard)
#         print("[logistics_score] =", logistics_score)
#         print("[logistics_is_standard] =", logistics_is_standard)
#         print("[aftersale_score] =", aftersale_score)
#         print("[aftersale_is_standard] =", aftersale_is_standard)
#         print("##" * 40)


#         xpath2_2_1 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[1]//p[@class='ebase-TooltipContent__value']/text()"
#         m_consult_score = selector.xpath(xpath2_2_1).extract() and selector.xpath(xpath2_2_1).extract()[0] or 0
#         xpath2_2_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[2]//p[@class='ebase-TooltipContent__value']/text()"
#         m_logistics_score = selector.xpath(xpath2_2_2).extract() and selector.xpath(xpath2_2_2).extract()[0] or 0
#         xpath2_2_3 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[3]//p[@class='ebase-TooltipContent__value']/text()"
#         m_aftersale_score = selector.xpath(xpath2_2_3).extract() and selector.xpath(xpath2_2_3).extract()[0] or 0
#         xpath2_2_4 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[4]//p[@class='ebase-TooltipContent__value']/text()"
#         m_complaints_score = selector.xpath(xpath2_2_4).extract() and selector.xpath(xpath2_2_4).extract()[0] or 0
#         xpath2_2_5 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[5]//p[@class='ebase-TooltipContent__value']/text()"
#         m_commodity_score = selector.xpath(xpath2_2_5).extract() and selector.xpath(xpath2_2_5).extract()[0] or 0


#         print("##" * 40)
#         print("[m_consult_score] =", m_consult_score)
#         print("[m_logistics_score] =", m_logistics_score)
#         print("[m_aftersale_score] =", m_aftersale_score)
#         print("[m_complaints_score] =", m_complaints_score)
#         print("[m_commodity_score] =", m_commodity_score)
#         print("##" * 40)


#         value = (store_name, count_date, ranking, hierarchy, industry, wyg_integration_score, wyg_is_standard, market_comprehensive_score, market_is_standard, 
#                  aftersale_score, aftersale_is_standard, logistics_score, logistics_is_standard,complaints_score , complaints_is_standard, 
#                  commodity_score, commodity_is_standard, m_aftersale_score, m_logistics_score, m_complaints_score, m_commodity_score,
#                  m_consult_score)

#         sql = """INSERT INTO ds_ali_shop_wyg_d(store_name, count_date, ranking, hierarchy, industry, wyg_integration_score, wyg_is_standard, market_comprehensive_score, market_is_standard, 
#                  aftersale_score, aftersale_is_standard, logistics_score, logistics_is_standard,complaints_score , complaints_is_standard, 
#                  commodity_score, commodity_is_standard, m_aftersale_score, m_logistics_score, m_complaints_score, m_commodity_score,
#                  m_consult_score)VALUES(%s,%s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s, %s,%s,%s, %s,%s,%s, %s, %s,%s,%s)"""
        
        
#         try:
#             cursor.execute(sql, value)
#         except Exception as e:
#             print(e)
        
#         db.commit()
        
        
#         scroll_to(browser,y=0)
        
#         # 重新加载
#         element2 = get_element(browser=browser, by=By.XPATH, value="//a[@class='ebase-Selector__header']")
#         js_click = """arguments[0].click();"""
#         browser.execute_script(js_click, element2)
        
#         elements = get_element(browser=browser, by=By.XPATH, value="//div[@class='ebase-Selector__menu ebase-Selector__open']//div[@class='legacy-oui-typeahead-menu']/span", multi=True)
        
#         ind += 1
        
#         if index == end:
#             break


cursor.close() #关闭连接
db.close()


In [5]:
browser.quit()

In [ ]:
# db = connect()
# cursor = db.cursor()

# # 获取服务视窗的数据
# page_source = browser.page_source.encode("utf-8")
# # 构建选择器
# selector1 = Selector(text=page_source)


# # 关闭弹窗
# if selector1.xpath("//div[@class='ebase-ImageTips__dsImageTipsWrapper']"):
#     close_popup = get_element(browser=browser, by=By.XPATH, value="//div[@class='ebase-ImageTips__dsImageTipsWrapper']//span[@class='ebase-ImageTips__dsImageTipsCloseBtn']")
#     action = ActionChains(browser)
#     action.click(close_popup).perform()


# time.sleep(2)
# # 获取服务视窗的数据
# page_source = browser.page_source.encode("utf-8")
# # 构建选择器
# selector = Selector(text=page_source)

# ranking = selector.xpath("//div[@class='month-rank main-index']//span[contains(@class, 'index-value')]/text()").extract() and selector.xpath("//div[@class='month-rank main-index']//span[contains(@class, 'index-value')]/text()").extract()[0].strip() or ""

# hierarchy =  selector.xpath("//div[@class='month-rank main-index']//span[@class='detail']/text()").extract() and selector.xpath("//div[@class='month-rank main-index']//span[@class='detail']/text()").extract()[0].strip() or ""

# industry = selector.xpath("//div[@class='trend-container']//p[@class='cate-name text-overflow']/text()").extract() and selector.xpath("//div[@class='trend-container']//p[@class='cate-name text-overflow']/text()").extract()[0].replace("行业：", '').strip() or ""

# print("##" * 40)
# print("[ranking] =", ranking)
# print("[hierarchy] =", hierarchy)
# print("[industry] =", industry)
# print("##" * 40)


# # 点击服务视窗
# element = get_element(browser=browser, by=By.XPATH, value="//div[@class='ebase-Switch__root op-ebase-switch']/span[2]")
# js_click = """arguments[0].click();"""
# browser.execute_script(js_click, element)


# # 浏览器的获取数据节点
# ele_shichuang = get_element(browser=browser, by=By.XPATH, value="//div[@id='floor-service-experience']")
# x, y = ele_shichuang.location.get("x"), ele_shichuang.location.get("y")

# # x, y = get_scroll_range(browser)
# scroll_to(browser,y=y)

# time.sleep(2)
# # 获取服务视窗的数据
# page_source = browser.page_source.encode("utf-8")
# # 构建选择器
# selector2 = Selector(text=page_source)

# if selector2.xpath("//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper']"):

#     # 需要鼠标放入到区域内
#     polygon_buttons = get_element(browser=browser, by=By.XPATH, value="//div[@class='componentsServiceExperienceBoard__root']//div[@class='componentsServiceExperienceBoard__radarChart']", multi=True)
#     for polygon_button in polygon_buttons:
#         action = ActionChains(browser)

# #                 x, y, width, heigth = polygon_button.location.get('x'), polygon_button.location.get('y'), polygon_button.size.get('width'), polygon_button.size.get('height')
#         action.move_to_element(polygon_button).perform()
#         time.sleep(1)

# # 获取服务视窗的数据
# page_source = browser.page_source.encode("utf-8")
# # 构建选择器
# selector = Selector(text=page_source)


# store_name = store_name
# count_date = selector.xpath("//div[@class='oui-date-picker-current-date']").extract() and selector.xpath("//div[@class='oui-date-picker-current-date']").extract()[0] or ""
# count_date = clear_html_re(count_date).replace("统计时间", "")


# xpath_wyg = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@value='wygDsrRmkScore']//div[@class='oui-indexCell-indexValue oui-num']/text()"
# wyg_integration_score = selector.xpath(xpath_wyg).extract() and selector.xpath(xpath_wyg).extract()[0] or 0.00
# xpath_is_stand = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='oui-indexCell-indexName']//span[contains(@class, 'componentsServiceExperienceBoard')]/text()"
# wyg_is_standard = selector.xpath(xpath_is_stand).extract() and selector.xpath(xpath_is_stand).extract()[0] or ""

# print("##" * 40)
# print("[count_date] =", count_date)
# print("[wyg_integration_score] =", wyg_integration_score)
# print("[wyg_is_standard] =", wyg_is_standard)
# print("##" * 40)

# xpath_market = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='oui-indexCell-indexValue oui-num']/text()"
# market_comprehensive_score = selector.xpath(xpath_market).extract() and selector.xpath(xpath_market).extract()[0] or 0.00
# xpath_is_market = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='oui-indexCell-indexName']//span[contains(@class, 'componentsServiceExperienceBoard')]/text()"
# market_is_standard = selector.xpath(xpath_is_market).extract() and selector.xpath(xpath_is_market).extract()[0] or ""

# print("##" * 40)
# print("[market_comprehensive_score] =", market_comprehensive_score)
# print("[market_is_standard] =", market_is_standard)
# print("##" * 40)

# xpath1 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//ul[@class='ebase-TooltipContent__legendContentList']/li[4]/p[@class='ebase-TooltipContent__value']/text()"
# xpath1_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='recharts-wrapper']//g[@class='recharts-layer recharts-polar-angle-axis']//g[@class='recharts-layer recharts-polar-angle-axis-ticks']/g[@class='recharts-layer recharts-polar-angle-axis-tick'][1]//text[contains(@class, 'componentsRadarTick')]/text()"

# # 商品体验
# commodity_score = selector.xpath(xpath1).extract() and selector.xpath(xpath1).extract()[0] or 0
# commodity_is_standard = selector.xpath(xpath1_2).extract() and selector.xpath(xpath1_2).extract()[0] or ""


# xpath2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//ul[@class='ebase-TooltipContent__legendContentList']/li[3]/p[@class='ebase-TooltipContent__value']/text()"
# xpath2_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='recharts-wrapper']//g[@class='recharts-layer recharts-polar-angle-axis']//g[@class='recharts-layer recharts-polar-angle-axis-ticks']/g[@class='recharts-layer recharts-polar-angle-axis-tick'][2]//text[contains(@class, 'componentsRadarTick')]/text()"

# # 投诉体验
# complaints_score = selector.xpath(xpath2).extract() and selector.xpath(xpath2).extract()[0] or 0
# complaints_is_standard = selector.xpath(xpath2_2).extract() and selector.xpath(xpath2_2).extract()[0] or ""


# xpath3 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//ul[@class='ebase-TooltipContent__legendContentList']/li[2]/p[@class='ebase-TooltipContent__value']/text()"
# xpath3_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='recharts-wrapper']//g[@class='recharts-layer recharts-polar-angle-axis']//g[@class='recharts-layer recharts-polar-angle-axis-ticks']/g[@class='recharts-layer recharts-polar-angle-axis-tick'][3]//text[contains(@class, 'componentsRadarTick')]/text()"

# # 物流体验
# logistics_score = selector.xpath(xpath3).extract() and selector.xpath(xpath3).extract()[0] or 0
# logistics_is_standard = selector.xpath(xpath3_2).extract() and selector.xpath(xpath3_2).extract()[0] or ""

# xpath4 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//ul[@class='ebase-TooltipContent__legendContentList']/li[1]/p[@class='ebase-TooltipContent__value']/text()"
# xpath4_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='recharts-wrapper']//g[@class='recharts-layer recharts-polar-angle-axis']//g[@class='recharts-layer recharts-polar-angle-axis-ticks']/g[@class='recharts-layer recharts-polar-angle-axis-tick'][4]//text[contains(@class, 'componentsRadarTick')]/text()"

# # 售后体验
# aftersale_score = selector.xpath(xpath4).extract() and selector.xpath(xpath4).extract()[0] or 0
# aftersale_is_standard = selector.xpath(xpath4_2).extract() and selector.xpath(xpath4_2).extract()[0] or ""

# print("##" * 40)
# print("[commodity_score] =", commodity_score)
# print("[commodity_is_standard] =", commodity_is_standard)
# print("[complaints_score] =", complaints_score)
# print("[complaints_is_standard] =", complaints_is_standard)
# print("[logistics_score] =", logistics_score)
# print("[logistics_is_standard] =", logistics_is_standard)
# print("[aftersale_score] =", aftersale_score)
# print("[aftersale_is_standard] =", aftersale_is_standard)
# print("##" * 40)


# xpath2_2_1 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[1]//p[@class='ebase-TooltipContent__value']/text()"
# m_consult_score = selector.xpath(xpath2_2_1).extract() and selector.xpath(xpath2_2_1).extract()[0] or 0
# xpath2_2_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[2]//p[@class='ebase-TooltipContent__value']/text()"
# m_logistics_score = selector.xpath(xpath2_2_2).extract() and selector.xpath(xpath2_2_2).extract()[0] or 0
# xpath2_2_3 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[3]//p[@class='ebase-TooltipContent__value']/text()"
# m_aftersale_score = selector.xpath(xpath2_2_3).extract() and selector.xpath(xpath2_2_3).extract()[0] or 0
# xpath2_2_4 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[4]//p[@class='ebase-TooltipContent__value']/text()"
# m_complaints_score = selector.xpath(xpath2_2_4).extract() and selector.xpath(xpath2_2_4).extract()[0] or 0
# xpath2_2_5 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[5]//p[@class='ebase-TooltipContent__value']/text()"
# m_commodity_score = selector.xpath(xpath2_2_5).extract() and selector.xpath(xpath2_2_5).extract()[0] or 0


# print("##" * 40)
# print("[m_consult_score] =", m_consult_score)
# print("[m_logistics_score] =", m_logistics_score)
# print("[m_aftersale_score] =", m_aftersale_score)
# print("[m_complaints_score] =", m_complaints_score)
# print("[m_commodity_score] =", m_commodity_score)
# print("##" * 40)


# value = (store_name, count_date, ranking, hierarchy, industry, wyg_integration_score, wyg_is_standard, market_comprehensive_score, market_is_standard, 
#          aftersale_score, aftersale_is_standard, logistics_score, logistics_is_standard,complaints_score , complaints_is_standard, 
#          commodity_score, commodity_is_standard, m_aftersale_score, m_logistics_score, m_complaints_score, m_commodity_score,
#          m_consult_score)

# sql = """INSERT INTO ds_ali_shop_wyg_d(store_name, count_date, ranking, hierarchy, industry, wyg_integration_score, wyg_is_standard, market_comprehensive_score, market_is_standard, 
#          aftersale_score, aftersale_is_standard, logistics_score, logistics_is_standard,complaints_score , complaints_is_standard, 
#          commodity_score, commodity_is_standard, m_aftersale_score, m_logistics_score, m_complaints_score, m_commodity_score,
#          m_consult_score)VALUES(%s,%s, %s, %s, %s, %s, %s, %s, %s, %s,%s,%s, %s,%s,%s, %s,%s,%s, %s, %s,%s,%s)"""


# try:
#     cursor.execute(sql, value)
# except Exception as e:
#     print(e)

# db.commit()

In [ ]:
# # 获取服务视窗的数据
# page_source = browser.page_source.encode("utf-8")
# # 构建选择器
# selector = Selector(text=page_source)


# store_name = store_name
# count_date = selector.xpath("//div[@class='oui-date-picker-current-date']").extract() and selector.xpath("//div[@class='oui-date-picker-current-date']").extract()[0] or ""
# count_date = clear_html_re(count_date).replace("统计时间", "")

# xpath_wyg = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@value='wygDsrRmkScore']//div[@class='oui-indexCell-indexValue oui-num']/text()"
# wyg_integration_score = selector.xpath(xpath_wyg).extract() and selector.xpath(xpath_wyg).extract()[0] or 0.00
# xpath_is_stand = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='oui-indexCell-indexName']//span[contains(@class, 'componentsServiceExperienceBoard')]/text()"
# wyg_is_standard = selector.xpath(xpath_is_stand).extract() and selector.xpath(xpath_is_stand).extract()[0] or ""

# print("##" * 40)
# print("[count_date] =", count_date)
# print("[wyg_integration_score] =", wyg_integration_score)
# print("[wyg_is_standard] =", wyg_is_standard)
# print("##" * 40)

# xpath_market = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='oui-indexCell-indexValue oui-num']/text()"
# market_comprehensive_score = selector.xpath(xpath_market).extract() and selector.xpath(xpath_market).extract()[0] or 0.00
# xpath_is_market = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='oui-indexCell-indexName']//span[contains(@class, 'componentsServiceExperienceBoard')]/text()"
# market_is_standard = selector.xpath(xpath_is_market).extract() and selector.xpath(xpath_is_market).extract()[0] or ""

# print("##" * 40)
# print("[market_comprehensive_score] =", market_comprehensive_score)
# print("[market_is_standard] =", market_is_standard)
# print("##" * 40)

# xpath1 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//ul[@class='ebase-TooltipContent__legendContentList']/li[4]/p[@class='ebase-TooltipContent__value']/text()"
# xpath1_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='recharts-wrapper']//g[@class='recharts-layer recharts-polar-angle-axis']//g[@class='recharts-layer recharts-polar-angle-axis-ticks']/g[@class='recharts-layer recharts-polar-angle-axis-tick'][1]//text[contains(@class, 'componentsRadarTick')]/text()"

# # 商品体验
# commodity_score = selector.xpath(xpath1).extract() and selector.xpath(xpath1).extract()[0] or 0
# commodity_is_standard = selector.xpath(xpath1_2).extract() and selector.xpath(xpath1_2).extract()[0] or ""


# xpath2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//ul[@class='ebase-TooltipContent__legendContentList']/li[3]/p[@class='ebase-TooltipContent__value']/text()"
# xpath2_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='recharts-wrapper']//g[@class='recharts-layer recharts-polar-angle-axis']//g[@class='recharts-layer recharts-polar-angle-axis-ticks']/g[@class='recharts-layer recharts-polar-angle-axis-tick'][2]//text[contains(@class, 'componentsRadarTick')]/text()"

# # 投诉体验
# complaints_score = selector.xpath(xpath2).extract() and selector.xpath(xpath2).extract()[0] or 0
# complaints_is_standard = selector.xpath(xpath2_2).extract() and selector.xpath(xpath2_2).extract()[0] or ""


# xpath3 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//ul[@class='ebase-TooltipContent__legendContentList']/li[2]/p[@class='ebase-TooltipContent__value']/text()"
# xpath3_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='recharts-wrapper']//g[@class='recharts-layer recharts-polar-angle-axis']//g[@class='recharts-layer recharts-polar-angle-axis-ticks']/g[@class='recharts-layer recharts-polar-angle-axis-tick'][3]//text[contains(@class, 'componentsRadarTick')]/text()"

# # 物流体验
# logistics_score = selector.xpath(xpath3).extract() and selector.xpath(xpath3).extract()[0] or 0
# logistics_is_standard = selector.xpath(xpath3_2).extract() and selector.xpath(xpath3_2).extract()[0] or ""

# xpath4 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//ul[@class='ebase-TooltipContent__legendContentList']/li[1]/p[@class='ebase-TooltipContent__value']/text()"
# xpath4_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][1]//div[@class='recharts-wrapper']//g[@class='recharts-layer recharts-polar-angle-axis']//g[@class='recharts-layer recharts-polar-angle-axis-ticks']/g[@class='recharts-layer recharts-polar-angle-axis-tick'][4]//text[contains(@class, 'componentsRadarTick')]/text()"

# # 售后体验
# aftersale_score = selector.xpath(xpath4).extract() and selector.xpath(xpath4).extract()[0] or 0
# aftersale_is_standard = selector.xpath(xpath4_2).extract() and selector.xpath(xpath4_2).extract()[0] or ""

# print("##" * 40)
# print("[commodity_score] =", commodity_score)
# print("[commodity_is_standard] =", commodity_is_standard)
# print("[complaints_score] =", complaints_score)
# print("[complaints_is_standard] =", complaints_is_standard)
# print("[logistics_score] =", logistics_score)
# print("[logistics_is_standard] =", logistics_is_standard)
# print("[aftersale_score] =", aftersale_score)
# print("[aftersale_is_standard] =", aftersale_is_standard)
# print("##" * 40)


# xpath2_2_1 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[1]//p[@class='ebase-TooltipContent__value']/text()"
# m_consult_score = selector.xpath(xpath2_2_1).extract() and selector.xpath(xpath2_2_1).extract()[0] or 0
# xpath2_2_2 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[2]//p[@class='ebase-TooltipContent__value']/text()"
# m_logistics_score = selector.xpath(xpath2_2_2).extract() and selector.xpath(xpath2_2_2).extract()[0] or 0
# xpath2_2_3 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[3]//p[@class='ebase-TooltipContent__value']/text()"
# m_aftersale_score = selector.xpath(xpath2_2_3).extract() and selector.xpath(xpath2_2_3).extract()[0] or 0
# xpath2_2_4 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[4]//p[@class='ebase-TooltipContent__value']/text()"
# m_complaints_score = selector.xpath(xpath2_2_4).extract() and selector.xpath(xpath2_2_4).extract()[0] or 0
# xpath2_2_5 = "//div[@class='componentsServiceExperienceBoard__root']//div[@class='home-card-module home-card-module-6 componentsServiceExperienceBoard__wygWrapper'][2]//div[@class='componentsServiceExperienceBoard__radarChart']//div[@class='recharts-tooltip-wrapper']//ul[@class='ebase-TooltipContent__legendContentList']/li[5]//p[@class='ebase-TooltipContent__value']/text()"
# m_commodity_score = selector.xpath(xpath2_2_5).extract() and selector.xpath(xpath2_2_5).extract()[0] or 0


# print("##" * 40)
# print("[m_consult_score] =", m_consult_score)
# print("[m_logistics_score] =", m_logistics_score)
# print("[m_aftersale_score] =", m_aftersale_score)
# print("[m_complaints_score] =", m_complaints_score)
# print("[m_commodity_score] =", m_commodity_score)
# print("##" * 40)